---
title: STEREO Magnetic field data pipeline
---

STEREO magnetic field is already in RTN coordinates, so no need to transform it.

Download data using `pyspedas`, but load it using `pycdfpp` (using `pyspedas` to load the data directly into `xarray` is very slow)

In [ ]:
#| hide
#| default_exp pipelines/stereo/mag
%load_ext autoreload
%autoreload 2

In [ ]:
from typing import Iterable

## Downloading data

In [ ]:
#| export
import os
os.environ['SPEDAS_DATA_DIR'] = f"{os.environ['HOME']}/data"
import pyspedas

In [ ]:
# | export

def download_mag_data(
    start: str = None,
    end: str = None,
    probe: str = "a",
) -> Iterable[str]:
    trange = [start, end]
    files = pyspedas.stereo.mag(trange, downloadonly=True, probe=probe)
    return files

## Preprocessing data

In [ ]:
#| export
from ids_finder.utils.basic import cdf2pl
from pipe import select
from typing import Iterable

In [ ]:
# | export
def preprocess_mag_data(
    raw_data: Iterable[str] = None, # List of CDF files
    ts: str = "1s",  # time resolution
) -> pl.DataFrame:
    """
    Preprocess the raw dataset (only minor transformations)

    - Downsample the data to a given time resolution
    - Applying naming conventions for columns
    """
    every = pandas.Timedelta(ts)
    period = 2 * every

    df: pl.LazyFrame = pl.concat(raw_data | pmap(cdf2pl, var_name="BFIELD"))

    return (
        df.pipe(resample, every=every, period=period)
        .rename(
            {
                "BFIELD_0": "b_r",
                "BFIELD_1": "b_t",
                "BFIELD_2": "b_n",
                "BFIELD_3": "b_mag",
            }
        )
        .collect()
    )

## Processing data

In [ ]:
#| export
def process_mag_data(
    raw_data: pl.DataFrame,
    ts: str = None,  # time resolution
    coord: str = None,
) -> Dict[str, pl.DataFrame]:
    """
    Corresponding to primary data layer, where source data models are transformed into domain data models

    - Partitioning data, for the sake of memory
    """
    return partition_data_by_year(raw_data)

### Pipeline

In [ ]:
#| export
def create_mag_data_pipeline(
    sat_id,
    ts: str = "1s",  # time resolution,
    tau: str = "60s",  # time window
    **kwargs,
) -> Pipeline:
    node_download_data = node(
        download_mag_data,
        inputs=dict(
            start="params:start_date",
            end="params:end_date",
        ),
        outputs=f"raw_mag_files",
        name=f"download_{sat_id.upper()}_magnetic_field_data",
    )

    node_preprocess_data = node(
        preprocess_mag_data,
        inputs=dict(
            raw_data=f"raw_mag_files",
        ),
        outputs=f"inter_mag_{ts}",
        name=f"preprocess_{sat_id.upper()}_magnetic_field_data",
    )

    node_process_data = node(
        process_mag_data,
        inputs=f"inter_mag_{ts}",
        outputs=f"primary_mag_{ts}",
        name=f"process_{sat_id.upper()}_magnetic_field_data",
    )

    node_extract_features = node(
        extract_features,
        inputs=[f"primary_mag_{ts}", "params:tau", "params:extract_params"],
        outputs=f"feature_tau_{tau}",
        name=f"extract_{sat_id}_features",
    )

    nodes = [
        node_download_data,
        node_preprocess_data,
        node_process_data,
        node_extract_features,
    ]

    pipelines = pipeline(
        nodes,
        namespace=sat_id,
        parameters={
            "params:start_date": "params:jno_start_date",
            "params:end_date": "params:jno_end_date",
            "params:tau": "params:tau",
        },
    )

    return pipelines